## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
# answer below:
wine.isnull().sum()


fixed.acidity           0
volatile.acidity        0
citric.acid             0
residual.sugar          0
chlorides               0
free.sulfur.dioxide     0
total.sulfur.dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [4]:
wine.quality.unique()

array([5, 6, 7, 4, 8, 3])

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [5]:
# answer below

wine.quality = np.where(wine.quality > 5, 1, 0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


Create a train test split with 20% of the data in the test subsample.

In [6]:
# answer below:

from sklearn.model_selection import train_test_split

y = wine.quality

X = wine.drop('quality', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [7]:
y

1       0
2       0
3       0
4       1
5       0
       ..
1595    0
1596    1
1597    1
1598    0
1599    1
Name: quality, Length: 1599, dtype: int64

Scale only the independent variables using the minmax scaler.

In [8]:
# answer below:

from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler()

scale.fit(X_train)

X_train_scale = scale.transform(X_train)

X_test_scale = scale.transform(X_test)


Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [9]:
# answer below:

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_scale, y_train)
print(
    f'Train accuracy: {knn.score(X_train_scale, y_train)}\n'
    f'Test accuracy: {knn.score(X_test_scale, y_test)}'
)



Train accuracy: 0.8193901485535575
Test accuracy: 0.73125


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [10]:
# answer below:

def one_over_sqrt_dist(dist):
    if dist.dtype is np.dtype(object):
        for point_dist_i, point_dist in enumerate(dist):
            if hasattr(point_dist, '__contains__') and 0. in point_dist:
                dist[point_dist_i] = point_dist == 0.
            else:
                dist[point_dist_i] = 1./np.sqrt(point_dist)
    else:
        with np.errstate(divide='ignore'):
            dist = 1./np.sqrt(dist)
            
            
        inf_mask = np.isinf(dist)
        inf_row = np.any(inf_mask, axis=1)
        dist[inf_row] = inf_mask[inf_row]
        
    return dist
    


In [11]:
knn2 = KNeighborsClassifier(n_neighbors=5, weights=one_over_sqrt_dist)

knn2.fit(X_train_scale, y_train)
print(
    f'Train accuracy: {knn2.score(X_train_scale, y_train)}\n'
    f'Test accuracy: {knn2.score(X_test_scale, y_test)}'
)

Train accuracy: 1.0
Test accuracy: 0.75625


In [12]:
from sklearn.metrics import classification_report

print(classification_report(y_test, knn2.predict(X_test_scale)))

              precision    recall  f1-score   support

           0       0.74      0.73      0.73       148
           1       0.77      0.78      0.77       172

    accuracy                           0.76       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.76      0.76      0.76       320



In [13]:
print(classification_report(y_train, knn2.predict(X_train_scale)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       596
           1       1.00      1.00      1.00       683

    accuracy                           1.00      1279
   macro avg       1.00      1.00      1.00      1279
weighted avg       1.00      1.00      1.00      1279

